In [1]:
import pandas as pd
from os.path import join

In [2]:
data_path = "../Dataset-1/selfie_dataset.txt"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "../Dataset-1/images"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [3]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [5]:
image_names = df_image_details.image_name.values.copy()
image_scores = df_image_details[headers[1]].values.copy()
image_attrs = df_image_details[headers[2:]].values.copy()

In [6]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [7]:
image_paths_train, image_paths_test = image_paths[:-1000], image_paths[-1000:]
image_attrs_train, image_attrs_test = image_attrs[:-1000], image_attrs[-1000:]
image_scores_train, image_scores_test = image_scores[:-1000], image_scores[-1000:]

In [8]:
from keras.utils import Sequence
import numpy as np
import cv2

Using TensorFlow backend.


In [9]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y1 = self.y[0][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y2 = self.y[1][idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y1 = [atrributes for atrributes in batch_y1]
        y2 = [atrributes for atrributes in batch_y2]
        return [np.array(x), np.array(x)], [np.array(y1), np.array(y2)]
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_CUBIC)
        return im / 255.

# Training Model


In [14]:
from keras.applications import resnet50
from keras.layers import Dense, Conv2D, MaxPool2D, Input, Flatten, concatenate, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [12]:
model_rnet = load_model("resnet50.hdf5")

/home/npc/anaconda3/envs/py36/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
for layer in model_rnet.layers:
    layer.trainable = False

In [15]:
model_classification = Dense(1024, activation='relu')(model_rnet.get_layer('avg_pool').output)
model_classification = Dropout(0.5)(model_classification)

model_classification = Dense(512, activation='relu')(model_classification)

model_classification = Dense(36, activation='sigmoid', name='classification')(model_classification)

In [33]:
model_regression_input = Input((224, 224, 3), name='input_regression')
model_regression = Conv2D(16, 3)(model_regression_input)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(24, 5)(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(32, 5)(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Flatten()(model_regression)
model_regression = Dense(128)(model_regression)

model_regression = concatenate([model_regression, model_classification])

model_regression = Dense(1, name='regression')(model_regression)

In [34]:
model = Model(inputs=[model_rnet.input, model_regression_input], outputs=[model_classification, model_regression])

In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [36]:
model.compile(
    optimizer='adam',
    loss={
        'regression': 'mean_squared_error',
        'classification': 'binary_crossentropy'
    },
    metrics=[
        'accuracy'
    ]
)

In [37]:
train_gen = ImageGenerator(image_paths_train, (image_attrs_train, image_scores_train), batch_size=128)
test_gen = ImageGenerator(image_paths_test, (image_attrs_test, image_scores_test), batch_size=128)

In [38]:
train_len = len(image_paths_train)
test_len = len(image_paths_test)
train_len, test_len

(44227, 1000)

In [39]:
model.fit_generator(train_gen, validation_data=test_gen, epochs=200, 
                    steps_per_epoch=train_len // 128,
                   validation_steps=10, use_multiprocessing=False,
                   callbacks=[
                       ReduceLROnPlateau(patience=2, verbose=1),
                       ModelCheckpoint('chpt-4_2.hdf5', verbose=1, save_best_only=True)
                   ])

Epoch 1/200
345/345 [==============================] - 270s 781ms/step - loss: 12.5174 - classification_loss: 0.4885 - regression_loss: 12.0289 - classification_acc: 0.8355 - regression_acc: 4.3026e-04 - val_loss: 0.8073 - val_classification_loss: 0.3461 - val_regression_loss: 0.4612 - val_classification_acc: 0.8316 - val_regression_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.80734, saving model to chpt-4_2.hdf5
Epoch 2/200
345/345 [==============================] - 263s 764ms/step - loss: 0.5436 - classification_loss: 0.3002 - regression_loss: 0.2434 - classification_acc: 0.8759 - regression_acc: 5.2084e-04 - val_loss: 0.6198 - val_classification_loss: 0.3069 - val_regression_loss: 0.3129 - val_classification_acc: 0.8852 - val_regression_acc: 0.0000e+00

Epoch 00002: val_loss improved from 0.80734 to 0.61983, saving model to chpt-4_2.hdf5
Epoch 3/200
345/345 [==============================] - 264s 765ms/step - loss: 0.5086 - classification_loss: 0.2881 - regression_l


Epoch 00019: val_loss did not improve from 0.57632
Epoch 20/200
345/345 [==============================] - 258s 749ms/step - loss: 0.4382 - classification_loss: 0.2665 - regression_loss: 0.1717 - classification_acc: 0.8866 - regression_acc: 5.8877e-04 - val_loss: 0.6172 - val_classification_loss: 0.3276 - val_regression_loss: 0.2896 - val_classification_acc: 0.8860 - val_regression_acc: 0.0016

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-11.

Epoch 00020: val_loss did not improve from 0.57632
Epoch 21/200
345/345 [==============================] - 258s 746ms/step - loss: 0.4370 - classification_loss: 0.2653 - regression_loss: 0.1717 - classification_acc: 0.8873 - regression_acc: 6.3406e-04 - val_loss: 0.6191 - val_classification_loss: 0.3342 - val_regression_loss: 0.2850 - val_classification_acc: 0.8764 - val_regression_acc: 7.9618e-04

Epoch 00021: val_loss did not improve from 0.57632
Epoch 22/200
345/345 [==============================] - 258s 748ms/

345/345 [==============================] - 259s 750ms/step - loss: 0.4377 - classification_loss: 0.2667 - regression_loss: 0.1710 - classification_acc: 0.8867 - regression_acc: 6.3406e-04 - val_loss: 0.6163 - val_classification_loss: 0.3319 - val_regression_loss: 0.2843 - val_classification_acc: 0.8766 - val_regression_acc: 7.9618e-04

Epoch 00038: ReduceLROnPlateau reducing learning rate to 1.000000032889008e-20.

Epoch 00038: val_loss did not improve from 0.57632
Epoch 39/200
345/345 [==============================] - 259s 749ms/step - loss: 0.4383 - classification_loss: 0.2652 - regression_loss: 0.1731 - classification_acc: 0.8873 - regression_acc: 6.1142e-04 - val_loss: 0.6191 - val_classification_loss: 0.3346 - val_regression_loss: 0.2844 - val_classification_acc: 0.8771 - val_regression_acc: 7.9618e-04

Epoch 00039: val_loss did not improve from 0.57632
Epoch 40/200
345/345 [==============================] - 258s 749ms/step - loss: 0.4369 - classification_loss: 0.2658 - regressio


Epoch 00056: ReduceLROnPlateau reducing learning rate to 1.0000001235416985e-29.

Epoch 00056: val_loss did not improve from 0.57632
Epoch 57/200
345/345 [==============================] - 258s 747ms/step - loss: 0.4389 - classification_loss: 0.2652 - regression_loss: 0.1737 - classification_acc: 0.8873 - regression_acc: 6.7935e-04 - val_loss: 0.6191 - val_classification_loss: 0.3342 - val_regression_loss: 0.2850 - val_classification_acc: 0.8764 - val_regression_acc: 7.9618e-04

Epoch 00057: val_loss did not improve from 0.57632
Epoch 58/200
345/345 [==============================] - 258s 748ms/step - loss: 0.4394 - classification_loss: 0.2683 - regression_loss: 0.1712 - classification_acc: 0.8858 - regression_acc: 6.3406e-04 - val_loss: 0.6163 - val_classification_loss: 0.3319 - val_regression_loss: 0.2843 - val_classification_acc: 0.8766 - val_regression_acc: 7.9618e-04

Epoch 00058: ReduceLROnPlateau reducing learning rate to 1.0000001536343539e-30.

Epoch 00058: val_loss did not i

345/345 [==============================] - 259s 750ms/step - loss: 0.4414 - classification_loss: 0.2681 - regression_loss: 0.1733 - classification_acc: 0.8860 - regression_acc: 5.2084e-04 - val_loss: 0.6191 - val_classification_loss: 0.3346 - val_regression_loss: 0.2844 - val_classification_acc: 0.8771 - val_regression_acc: 7.9618e-04

Epoch 00075: val_loss did not improve from 0.57632
Epoch 76/200
345/345 [==============================] - 259s 749ms/step - loss: 0.4361 - classification_loss: 0.2652 - regression_loss: 0.1709 - classification_acc: 0.8875 - regression_acc: 6.7935e-04 - val_loss: 0.6172 - val_classification_loss: 0.3276 - val_regression_loss: 0.2896 - val_classification_acc: 0.8860 - val_regression_acc: 0.0016

Epoch 00076: ReduceLROnPlateau reducing learning rate to 1.0000000751754869e-39.

Epoch 00076: val_loss did not improve from 0.57632
Epoch 77/200
345/345 [==============================] - 259s 752ms/step - loss: 0.4392 - classification_loss: 0.2668 - regression_l

KeyboardInterrupt: 